In [ ]:
import pandas as pd

In [ ]:
import tweepy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import datetime
import time

# Setting up Twitter API credentials

api_key = '5EHtW9kM9sNsM9YQYKotF3v6a'
api_key_secret = 'yf0jmTZAUDYPIaC5D8KwC41kmGwdwz6UoBemKlZf1XJlLwUTYA'
access_token = '1856301676166053889-B6h2ygJW8rJBoBHjpnOGsmX0ivw0aM'
access_token_secret = 'iGAd8LALDOQVBwnIcapXaIN1Dcr3St8CtcyyLFeLwDX3J'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMSzwwEAAAAABik0GwXyz5lQhNdWvDjw62rNl04%3DX3sdGdvbXx3MxN056Il3fAWUhn2IyJY2AIs1Ob8Ntz7ep8Sd47'

# Authenticate with Twitter
client = tweepy.Client(bearer_token=bearer_token)

# Initialize VADER sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to get tweet sentiment
def get_tweet_sentiment(tweet_text):
    score = analyzer.polarity_scores(tweet_text)
    return score['compound']

# Fetch tweets with a specified query and handle rate limit errors
def fetch_tweets(stock_symbol, start_date, end_date, max_results=10):
    query = f"{stock_symbol} -is:retweet"
    data = []
    
    while True:
        try:
            # Fetch tweets from the Twitter API
            tweets = client.search_recent_tweets(
                query=query,
                start_time=start_date,
                end_time=end_date,
                max_results=max_results,
                tweet_fields=['created_at', 'text', 'lang']
            )
            
            # Process each tweet
            for tweet in tweets.data:
                if tweet.lang == 'en':
                    sentiment = get_tweet_sentiment(tweet.text)
                    data.append({'date': tweet.created_at.date(), 'Tweet': tweet.text, 'Sentiment': sentiment})

            # Return data as a DataFrame
            return pd.DataFrame(data)
        
        except tweepy.TooManyRequests:
            print("Rate limit hit. Waiting 1 minutes before retrying...")
            time.sleep(1 * 60)  # Wait for 15 minutes and retry
        except Exception as e:
            print(f"An error occurred: {e}")
            break

# Examlpe usage
start_date = (datetime.datetime.now() - datetime.timedelta(days=7)).isoformat("T") + "Z" # 7 days ago
end_date = (datetime.datetime.now() - datetime.timedelta(days=1)).isoformat("T") + "Z"  #Current date
stock_symbol = "$SUZLON"

# Fetch tweets and calculate sentiments
tweet_df = fetch_tweets(stock_symbol, start_date, end_date)
# tweet_df.head()